In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import pytorch_lightning as pl
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F
import numpy as np
from ToxicityDataModule import ToxicityDataModule, max_token_len
from ToxicityClassifier import ToxicityClassifier
import json

KeyboardInterrupt: 

In [ ]:
data_path = "./dataset/processed/train.csv"
ds = pd.read_csv(data_path, dtype={"id": str})
ds.head

In [ ]:
X = ds["comment_text"]
y = ds["score"]
X, _, y, _ = train_test_split(X, y, test_size=0.7, random_state=103)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=103)
train_ds = pd.DataFrame({"comment_text": X_train, "score": y_train})
val_ds = pd.DataFrame({"comment_text": X_test, "score": y_test})

val_ds.head

In [ ]:
config = json.load(open("config.json"))

data_module = ToxicityDataModule(train_ds, val_ds, max_token_len, model_name = config["model_name"], batch_size = config["batch_size"])
data_module.setup()

In [ ]:
trainer = pl.Trainer(max_epochs=config["n_epochs"], num_sanity_val_steps=2, logger = True, enable_progress_bar = True, num_nodes = 1)
model = ToxicityClassifier(config)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
torch.set_float32_matmul_precision("medium")

model.load_state_dict(torch.load("./trained/v1.pth"))

In [ ]:
def predict_toxicity(model, data_module):
    predictions = trainer.predict(model, datamodule=data_module)
    flattened_predictions = np.stack([torch.sigmoid(torch.Tensor(p)) for batch in predictions for p in batch])
    return flattened_predictions

In [ ]:
predictions = predict_toxicity(model, data_module)
predictions

In [ ]:
predictions.shape

In [ ]:
for pred in predictions:
    print(pred)

In [ ]:
predictions = np.round(predictions)
predictions

In [ ]:
from sklearn import metrics
plt.figure(figsize=(15, 8))
for i, attribute in enumerate(attributes):
  fpr, tpr, _ = metrics.roc_curve(
      true_labels[:,i].astype(int), predictions[:, i])
  auc = metrics.roc_auc_score(
      true_labels[:,i].astype(int), predictions[:, i])
  plt.plot(fpr, tpr, label='%s %g' % (attribute, auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.title('RoBERTa Trained on UCC Datatset - AUC ROC')